In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpassword"

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
    result = session.run("MATCH (n) RETURN n")
    for record in result:
        print(record)
driver.close()

<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:0' labels=frozenset({'Person'}) properties={'name': 'Alice', 'age': 30}>>
<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:1' labels=frozenset({'Person'}) properties={'name': 'Bob', 'age': 25}>>
<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:2' labels=frozenset({'Person'}) properties={'name': 'Charlie', 'age': 35}>>
<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:3' labels=frozenset({'Station'}) properties={'latitude': -87.74919253254365, 'name': 'Oakton-Skokie', 'longitude': 'Yellow Line'}>>
<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:4' labels=frozenset({'Station'}) properties={'latitude': -87.67625887027687, 'name': 'Howard', 'longitude': 'Yellow Line'}>>
<Record n=<Node element_id='4:ca41dbad-5021-46c8-98bb-c521b336b04e:5' labels=frozenset({'Station'}) properties={'latitude': -87.7357419301787, 'name': 'Pulaski-Midway', 'longitude': 'Ora

# Menu

1) <a href="#insert">Insert ad-hoc data</a>
2) <a href="#csv">Load a CSV of data</a>
3) <a href="#query">Query database</a>

<a name="insert" />

# Insert ad-hoc data

In [2]:
from neo4j import GraphDatabase

# Connect to the Neo4j instance
uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpassword"

driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to add data
def add_data(session):
    session.run(
        """
        CREATE (a:Person {name: 'Alice', age: 30})
        CREATE (b:Person {name: 'Bob', age: 25})
        CREATE (c:Person {name: 'Charlie', age: 35})
        CREATE (a)-[:FRIENDS_WITH]->(b)
        CREATE (b)-[:FRIENDS_WITH]->(c)
        """
    )

# Function to query data
def query_data(session):
    result = session.run(
        """
        MATCH (p:Person)-[:FRIENDS_WITH]->(friend)
        RETURN p.name AS person, friend.name AS friend
        """
    )
    for record in result:
        print(f"{record['person']} is friends with {record['friend']}")

# Run the add and query functions in a session
with driver.session() as session:
    add_data(session)
    query_data(session)

# Close the driver connection
driver.close()


Alice is friends with Bob
Bob is friends with Charlie
Alice is friends with Bob
Bob is friends with Charlie


<a name="csv" />

# Load a CSV of data

In [3]:
import pandas as pd
nodes_data = pd.read_csv('../data/nodes_data.csv')
nodes_data.sample(5)

,Station,Lines,Longitude,Latitude
45,Montrose-Ravenswood,Brown Line,-87.676044,41.965402
103,Addison-O'Hare,Blue Line (O'Hare),-87.722231,41.949117
139,47th-South Elevated,Green Line,-87.618930,41.812752
78,54/Cermak,Pink Line,-87.750849,41.851873
51,Grand/State,Red Line,-87.628097,41.894090


In [4]:
import pandas as pd
edges_data = pd.read_csv('../data/edges_data.csv')
edges_data.sample(5)

,From,To,Type,Legend,Alt_Legend,Branch,Owl,Shape,Shape_len
78,Damen-Ravenswood,Montrose-Ravenswood,Elevated or at Grade,BR,BR,Brown Line,No,Polyline,2371.581205
88,Cermak-McCormick Place,17th Junction,Elevated or at Grade,GR,GR,Green Line South Elevated,No,Polyline,2057.480507
31,California-Lake,Paulina Junction,Elevated or at Grade,GR,GR,Green Line Lake Street,No,Polyline,7309.368408
30,Kedzie-Douglas,California-Douglas,Elevated or at Grade,PK,PK,Pink Line,No,Polyline,2889.800417
144,Clark/Lake,Washington/Dearborn,Subway,BL,BL,Blue Line Dearborn Subway,Yes,Polyline,1285.128506


<a name="insert" />

# Create a new Neo4j graph

In [5]:
from neo4j import GraphDatabase
import pandas as pd

uri = "bolt://localhost:7687"
username = "neo4j"
password = "testpassword"
driver = GraphDatabase.driver(uri, auth=(username, password))

<a name="insert" />

# Load the Nodes to the Graph

In [6]:
with driver.session() as session:
    for index, row in nodes_data.iterrows():
        session.run(
            "CREATE (:Station {name: $name, longitude: $long, latitude: $lat})",
            name=row[0], long=row[1], lat=row[2]
        )

<a name="insert" />

# Load the Edges to the Graph

In [7]:
with driver.session() as session:
    for index, row in edges_data.iterrows():
        session.run(
            """
            MATCH (from:Station {name: $from_node})
            MATCH (to:Station {name: $to_node})
            CREATE (from)-[:CONNECTED_TO {
                type: $type, 
                legend: $legend,
                alt_legend: $alt_legend,
                branch: $branch,
                owl: $owl,
                shape: $shape,
                shape_len: toFloat($shape_len)
            }]->(to)
            """,
            from_node=row['From'],
            to_node=row['To'],
            type=row['Type'],
            legend=row['Legend'],
            alt_legend=row['Alt_Legend'],
            branch=row['Branch'],
            owl=row['Owl'],            
            shape=row['Shape'],       
            shape_len=row['Shape_len']
        )

In [8]:
with driver.session() as session:
    node_count = session.run("MATCH (n:Station) RETURN COUNT(n)").single()[0]
    edge_count = session.run("MATCH ()-[r:CONNECTED_TO]->() RETURN COUNT(r)").single()[0]

print(f"Number of stations: {node_count}")
print(f"Number of connections: {edge_count}")

Number of stations: 306
Number of connections: 765


In [9]:
nodes_data.shape

(153, 4)

In [10]:
edges_data.shape

(153, 9)

<a name="query" />

# Query database

In [11]:
# What is the total number of stations in the graph?
with driver.session() as session:
    result = session.run(
        """
        MATCH (s:Station)
        RETURN COUNT(s) AS TotalStations
        """
    )
    total_stations = result.single()["TotalStations"]

print(f"Total number of stations in the graph: {total_stations}")

Total number of stations in the graph: 306


In [12]:
# Find the station with the most connections
with driver.session() as session:
    result = session.run(
        """
        MATCH (station:Station)-[:CONNECTED_TO]-()
        RETURN station.name AS StationName, COUNT(*) AS ConnectionCount
        ORDER BY ConnectionCount DESC
        LIMIT 1
        """
    )
    record = result.single()
    station_name = record["StationName"]
    connection_count = record["ConnectionCount"]

print(f"Station with the most connections: {station_name} ({connection_count} connections)")

Station with the most connections: Clark/Lake (20 connections)


In [13]:
# What are the top 10 branches by total connection distance?
with driver.session() as session:
    result = session.run(
        """
        MATCH (:Station)-[r:CONNECTED_TO]-(:Station)
        RETURN r.branch AS Branch, SUM(r.shape_len) AS TotalDistanceFeet
        ORDER BY TotalDistanceFeet DESC
        LIMIT 10
        """
    )
    top_branches = [
        (record["Branch"], round(record["TotalDistanceFeet"] / 5280, 2), round(record["TotalDistanceFeet"] / 3.281 / 1000, 2))
        for record in result
    ]

print("Top 10 branches by total connection distance:")
for branch, distance_miles, distance_km in top_branches:
    print(f"{branch}: {distance_miles} miles ({distance_km} km)")

Top 10 branches by total connection distance:
Blue Line O'Hare: 171.12 miles (275.37 km)
Red Line North Side: 104.94 miles (168.87 km)
Red Line Dan Ryan: 101.81 miles (163.84 km)
Blue Line Forest Park: 96.61 miles (155.47 km)
Orange Line: 91.1 miles (146.61 km)
Green Line Lake Street: 87.44 miles (140.71 km)
Brown Line: 73.92 miles (118.96 km)
Pink Line: 66.84 miles (107.56 km)
Green Line South Elevated: 64.8 miles (104.28 km)
Yellow Line: 49.29 miles (79.32 km)


In [14]:
# Find and display stations connected by the "Orange Line"
with driver.session() as session:
    result = session.run(
        """
        MATCH (a:Station)-[r:CONNECTED_TO {branch: 'Orange Line'}]-(b:Station)
        RETURN a.name AS FromStation, b.name AS ToStation
        ORDER BY FromStation, ToStation
        """
    )
    connections = [(record["FromStation"], record["ToStation"]) for record in result]

print("Stations connected by the 'Orange Line':")
for index, (from_station, to_station) in enumerate(connections, start=1):
    print(f"{index}. {from_station} -> {to_station}")

Stations connected by the 'Orange Line':
1. 17th Junction -> Halsted-Midway
2. 17th Junction -> Halsted-Midway
3. 17th Junction -> Halsted-Midway
4. 17th Junction -> Halsted-Midway
5. 17th Junction -> Halsted-Midway
6. 35th/Archer -> Ashland-Midway
7. 35th/Archer -> Ashland-Midway
8. 35th/Archer -> Ashland-Midway
9. 35th/Archer -> Ashland-Midway
10. 35th/Archer -> Ashland-Midway
11. 35th/Archer -> Western-Midway
12. 35th/Archer -> Western-Midway
13. 35th/Archer -> Western-Midway
14. 35th/Archer -> Western-Midway
15. 35th/Archer -> Western-Midway
16. Ashland-Midway -> 35th/Archer
17. Ashland-Midway -> 35th/Archer
18. Ashland-Midway -> 35th/Archer
19. Ashland-Midway -> 35th/Archer
20. Ashland-Midway -> 35th/Archer
21. Ashland-Midway -> Halsted-Midway
22. Ashland-Midway -> Halsted-Midway
23. Ashland-Midway -> Halsted-Midway
24. Ashland-Midway -> Halsted-Midway
25. Ashland-Midway -> Halsted-Midway
26. Halsted-Midway -> 17th Junction
27. Halsted-Midway -> 17th Junction
28. Halsted-Midway ->

In [15]:
with driver.session() as session:
    result = session.run("""
        MATCH (a:Station)-[r:CONNECTED_TO]-(b:Station)
        RETURN 
            a.name AS FromStation, 
            b.name AS ToStation, 
            r.shape_len/5280 AS DistanceMiles,  // Convert feet to miles
            r.shape_len/3.281 AS DistanceMeters  // Convert feet to meters
        ORDER BY r.shape_len DESC
        LIMIT 1
    """)
    record = result.single()
    from_station = record["FromStation"]
    to_station = record["ToStation"]
    distance_miles = round(record["DistanceMiles"], 2)
    distance_km = round(record["DistanceMeters"]/1000, 2)
    print(f"Longest connection is from {from_station} to {to_station}")
    print(f"Distance: {distance_miles} miles ({distance_km} km)")

Longest connection is from Howard to Oakton-Skokie
Distance: 4.01 miles (6.46 km)


In [16]:
# Find and display stations directly connected to "Howard"
with driver.session() as session:
    result = session.run(
        """
        MATCH (a:Station {name: 'Howard'})-[:CONNECTED_TO]-(b:Station)
        RETURN b.name AS ConnectedStation
        ORDER BY b.name
        """
    )
    connected_stations = [record["ConnectedStation"] for record in result]


print(f"Stations directly connected to 'Howard':")
for index, station in enumerate(connected_stations, start=1):
    print(f"{index}. {station}")

Stations directly connected to 'Howard':
1. Jarvis
2. Jarvis
3. Jarvis
4. Jarvis
5. Jarvis
6. Oakton-Skokie
7. Oakton-Skokie
8. Oakton-Skokie
9. Oakton-Skokie
10. Oakton-Skokie
11. South Blvd.
12. South Blvd.
13. South Blvd.
14. South Blvd.
15. South Blvd.


In [17]:
# Calculate the average distance of all connections
with driver.session() as session:
    result = session.run(
        """
        MATCH (:Station)-[r:CONNECTED_TO]-(:Station)
        RETURN 
            AVG(r.shape_len)/5280 AS AverageDistanceMiles,  // Convert feet to miles
            AVG(r.shape_len)/3.281 AS AverageDistanceMeters  // Convert feet to meters
        """
    )
    record = result.single()
    avg_distance_miles = round(record["AverageDistanceMiles"], 2)
    avg_distance_km = round(record["AverageDistanceMeters"]/1000, 2)

print(f"Average distance of all connections: {avg_distance_miles} miles ({avg_distance_km} km)")

Average distance of all connections: 0.67 miles (1.08 km)


In [18]:
# Function to find the shortest path between two stations
def find_shortest_path(driver, start_station, end_station):
    with driver.session() as session:
        result = session.run(
            """
            MATCH p=shortestPath((start:Station {name: $start_station})-[:CONNECTED_TO*]-(end:Station {name: $end_station}))
            RETURN [n IN nodes(p) | n.name] AS Path, length(p) AS Stops
            """,
            start_station=start_station,
            end_station=end_station
        )
        record = result.single()
        if record:
            path = record["Path"]
            stops = record["Stops"]
            print(f"Shortest route from '{start_station}' to '{end_station}':")
            print(f"  - Path: {' -> '.join(path)}")
            print()
            print(f"  - Total stops: {stops}")
        else:
            print(f"No path found between '{start_station}' and '{end_station}'.")

start_station = "Howard"  
end_station = "Belmont-O'Hare"  
find_shortest_path(driver, start_station, end_station)

Shortest route from 'Howard' to 'Belmont-O'Hare':
  - Path: Howard -> Jarvis -> Morse -> Loyola -> Granville -> Thorndale -> Bryn Mawr -> Berwyn -> Argyle -> Lawrence -> Wilson -> Sheridan -> Addison-North Main -> Clark Junction -> Belmont-North Main -> Wellington -> Diversey -> Fullerton -> Armitage -> Subway Portal B -> Sedgwick -> Chicago/Franklin -> Merchandise Mart -> Tower 18 -> Clark/Lake -> Grand/Milwaukee -> Chicago/Milwaukee -> Division/Milwaukee -> Damen/Milwaukee -> Western/Milwaukee -> California/Milwaukee -> Logan Square -> Belmont-O'Hare

  - Total stops: 32


/Users/dayvheed/opt/anaconda3/lib/python3.9/site-packages/neo4j/_sync/work/result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


In [19]:
# Find the station with the longest cumulative connections
with driver.session() as session:
    result = session.run(
        """
        MATCH (station:Station)-[r:CONNECTED_TO]-()
        RETURN 
            station.name AS StationName, 
            SUM(r.shape_len)/5280 AS TotalDistanceMiles,  // Convert feet to miles
            SUM(r.shape_len)/3.281 AS TotalDistanceMeters  // Convert feet to meters
        ORDER BY TotalDistanceMiles DESC
        LIMIT 1
        """
    )
    record = result.single()
    station_name = record["StationName"]
    total_distance_miles = round(record["TotalDistanceMiles"], 2)
    total_distance_km = round(record["TotalDistanceMeters"]/1000, 2)

print(f"Station with the longest cumulative connections:")
print(f"{station_name} with a total distance of {total_distance_miles} miles ({total_distance_km} km)")

Station with the longest cumulative connections:
Howard with a total distance of 24.99 miles (40.21 km)


In [20]:
# Determine which branch connects the most stations
with driver.session() as session:
    result = session.run(
        """
        MATCH (:Station)-[r:CONNECTED_TO]-(:Station)
        RETURN r.branch AS Branch, COUNT(DISTINCT r) AS UniqueConnections
        ORDER BY UniqueConnections DESC
        LIMIT 1
        """
    )
    record = result.single()
    branch = record["Branch"]
    connections = record["UniqueConnections"]

print(f"Branch connecting the most unique stations: {branch} ({connections} connections)")

Branch connecting the most unique stations: Red Line North Side (120 connections)


In [21]:
# Find stations reachable within 3 hops from "Howard"
with driver.session() as session:
    result = session.run(
        """
        MATCH (start:Station {name: 'Howard'})-[:CONNECTED_TO*..3]-(reachable:Station)
        RETURN DISTINCT reachable.name AS StationName
        ORDER BY reachable.name
        """
    )
    stations = [record["StationName"] for record in result]

if stations:
    print(f"Stations reachable within 3 hops from 'Howard' ({len(stations)} total):")
    for index, station in enumerate(stations, start=1):
        print(f"{index}. {station}")
else:
    print("No stations are reachable within 3 hops from 'Howard'.")

Stations reachable within 3 hops from 'Howard' (9 total):
1. Dempster
2. Dempster-Skokie
3. Howard
4. Jarvis
5. Loyola
6. Main
7. Morse
8. Oakton-Skokie
9. South Blvd.


In [22]:
# Find stations that belong to more than one branch
with driver.session() as session:
    result = session.run(
        """
        MATCH (station:Station)-[r:CONNECTED_TO]-()
        WITH station, COLLECT(DISTINCT r.branch) AS Branches
        WHERE size(Branches) > 1
        RETURN station.name AS StationName, Branches
        """
    )
    stations = [(record["StationName"], record["Branches"]) for record in result]

print(f"Stations part of multiple branches:")
for station_name, branches in stations:
    print(f"{station_name}: {', '.join(branches)}")

Stations part of multiple branches:
Howard: Purple Line Evanston, Red Line North Side, Yellow Line
Paulina Junction: Green Line Lake Street, Pink Line
Roosevelt/State: Red Line State Street Subway, Red Line Dan Ryan
Clark/Lake: Blue Line Dearborn Subway, Blue Line O'Hare, Loop Elevated
Subway Portal B: Red Line North Side, Brown Line
17th Junction: Green Line South Elevated, Orange Line
Tower 18: Loop Elevated, Brown Line, Green Line Lake Street
Lake/State: Red Line State Street Subway, Red Line North Side
59th Junction: Green Line East 63rd Branch, Green Line South Elevated, Green Line Ashland/63rd Branch
Tower 12: Loop Elevated, Green Line South Elevated
LaSalle: Blue Line Forest Park, Blue Line Dearborn Subway
Clark Junction: Red Line North Side, Brown Line
Howard: Purple Line Evanston, Red Line North Side, Yellow Line
Paulina Junction: Green Line Lake Street, Pink Line
Roosevelt/State: Red Line State Street Subway, Red Line Dan Ryan
Clark/Lake: Blue Line Dearborn Subway, Blue Line 

In [23]:
# Find stations that belong to more than one branch
with driver.session() as session:
    result = session.run(
        """
        MATCH (station:Station)-[r:CONNECTED_TO]-()
        WITH station, COLLECT(DISTINCT r.branch) AS Branches
        WHERE size(Branches) > 1
        RETURN station.name AS StationName, Branches
        ORDER BY station.name
        """
    )
    stations = [(record["StationName"], record["Branches"]) for record in result]

if stations:
    print(f"Stations that are part of multiple branches ({len(stations)} total):")
    for index, (station_name, branches) in enumerate(stations, start=1):
        branch_list = ', '.join(branches)
        print(f"{index}. {station_name}: Branches: [{branch_list}]")
else:
    print("No stations are part of multiple branches.")

Stations that are part of multiple branches (24 total):
1. 17th Junction: Branches: [Green Line South Elevated, Orange Line]
2. 17th Junction: Branches: [Green Line South Elevated, Orange Line]
3. 59th Junction: Branches: [Green Line East 63rd Branch, Green Line South Elevated, Green Line Ashland/63rd Branch]
4. 59th Junction: Branches: [Green Line East 63rd Branch, Green Line South Elevated, Green Line Ashland/63rd Branch]
5. Clark Junction: Branches: [Red Line North Side, Brown Line]
6. Clark Junction: Branches: [Red Line North Side, Brown Line]
7. Clark/Lake: Branches: [Blue Line Dearborn Subway, Blue Line O'Hare, Loop Elevated]
8. Clark/Lake: Branches: [Blue Line Dearborn Subway, Blue Line O'Hare, Loop Elevated]
9. Howard: Branches: [Purple Line Evanston, Red Line North Side, Yellow Line]
10. Howard: Branches: [Purple Line Evanston, Red Line North Side, Yellow Line]
11. LaSalle: Branches: [Blue Line Forest Park, Blue Line Dearborn Subway]
12. LaSalle: Branches: [Blue Line Forest Pa

In [24]:
# Find top 3 busiest stations by cumulative distance
with driver.session() as session:
    result = session.run(
        """
        MATCH (station:Station)-[r:CONNECTED_TO]-()
        RETURN 
            station.name AS StationName, 
            SUM(r.shape_len)/5280 AS TotalDistanceMiles,  // Convert feet to miles
            SUM(r.shape_len)/3.281 AS TotalDistanceMeters  // Convert feet to meters
        ORDER BY TotalDistanceMiles DESC
        LIMIT 3
        """
    )
    top_stations = [
        (
            record["StationName"],
            round(record["TotalDistanceMiles"], 2),
            round(record["TotalDistanceMeters"] / 1000, 2)
        )
        for record in result
    ]

print(f"Top 3 busiest stations by cumulative distance:")
for station_name, distance_miles, distance_km in top_stations:
    print(f"{station_name}: {distance_miles} miles ({distance_km} km)")

Top 3 busiest stations by cumulative distance:
Howard: 24.99 miles (40.21 km)
Oakton-Skokie: 24.64 miles (39.66 km)
Harlem-O'Hare: 21.22 miles (34.15 km)
